In [ ]:
N = 2 ** 4
M = N // 4

# Generate a deterministic DBBD matrix
A = generate_DBDD_matrix(M, N)
print('DBDD:')
printFormatted(A)

# Generate a random Gaussian matrix
B = generate_random_matrix(M, N, matrix_type='gaussian')
print('Gaussian:')
printFormatted(B)


# Generate a random scaled binary matrix
C = generate_random_matrix(M, N, matrix_type='scaled_binary')
print('Scaled binary:')
printFormatted(C)

# Generate a random unscaled binary matrix
D = generate_random_matrix(M, N, matrix_type='unscaled_binary')
print('Unscaled binary:')
printFormatted(D)


#### Generate dictionary

In [ ]:
DIM = 16

# Generate dictionary
A = generate_dct_dictionary(DIM)  # test what you want
print(f'Dictionary shape: {A.shape}')
print()

# Orthonormality test
print("Orthonormality test")

# Check orthogonality
rankA = np.linalg.matrix_rank(A)
#print(f'Rank of A: {rankA}')
if A.shape[0] == rankA or A.shape[1] == rankA:
    print("Matrix A is full rank.")

# Check normalization
is_normalized = check_normalization(A)
print(f"Matrix A is normalized: {is_normalized}")
is_normalized = check_normalization(A.T)
print(f"Matrix A^T is normalized: {is_normalized}")
print()

# Coherence test
print("Coherence test")
coherence = compute_coherence(A)
print(f'Coherence: {coherence}')


#### Code execution

In [ ]:
import scipy.io
import matplotlib.pyplot as plt

if __name__ == "__main__":

    
    # I put this here because you have to decide kron fact based on
    # the signal length, but for testing reasons I prefer to do the 
    # opposite: choose signal length based on kron factor
    
    n_block = 16 # block size, deafult is 16
    kron_factor = n_block * 2  # kron factor default is 32


    ## DATA
    # ----------------------------------------------------------------
    #  upload data
    data = scipy.io.loadmat('100m.mat')
    # retrieve the key to a string
    key = list(data.keys())[0]
    # retrieve the values
    signal = data[key][0,:]  # [0 or 1, 0:650000] s.t. first dim: (0 is MLII, 1 is V5)
    num = 10  # how many kronecker blocks will be long the original signals
    temp = n_block * kron_factor  # this is equal to the length of a kronecker block
    start = int(temp * 0)  # choose where to start our signal in the record (signal is a piece of the record)
    end = int(start + temp * num)
    signal = signal[start:end]  # comment out to use the whole signal
    # print the shape of the signal
    print(f'Signal shape: {signal.shape}') 
    # plot the signal
    plt.figure(figsize=(10, 6))
    plt.plot(signal)
    plt.title('ECG Signal')
    plt.xlabel('Index')
    plt.ylabel('Amplitude')
    plt.grid(True)
    plt.show()    

    ## PARAMETERS
    # ----------------------------------------------------------------
    CR = 1/4  # compression ratio
    # NON-KRONECKER
    #n_block = 16 # block size`
    m_block = int(n_block * CR) # compressed block size
    # KRONECKER
    #kron_factor = 32  # kron factor
    n_block_kron = n_block * kron_factor  # kron block size
    m_block_kron = int(n_block_kron * CR) # compressed kron block size
    


    ### SAMPLING PHASE 
    # ----------------------------------------------------------------
    # MEASUREMENT MATRIX
    Phi = generate_DBDD_matrix(m_block, n_block)
    #Phi = generate_random_matrix(m_block, n_block, matrix_type='gaussian')
    #Phi = generate_random_matrix(m_block, n_block, matrix_type='scaled_binary')
    #Phi = generate_random_matrix(m_block, n_block, matrix_type='unscaled_binary')

    # COMPRESS THE SIGNAL
    Y = compressSignal(signal, Phi)



    ## RECOVERY PHASE
    # ----------------------------------------------------------------
    # SL0 PARAMETERS
    sigma_off = 0.001  # offset for sigma
    mu_0 = 2  # scaling factor for mu
    sigma_decrease_factor = 0.5  # factor for decreasing sigma
    L = 3  # number of iterations for the inner loop
    if sigma_off > 0:
        sigma_min = sigma_off * 4  # minimum value of sigma
    else:
        sigma_min = 0.00001  # minimum value of sigmZ
    # RECOVERY non-KRONECKER
    recovered_signal = non_kron_recovery(Y=Y, sigma_min=sigma_min, Phi=Phi, sigma_decrease_factor=sigma_decrease_factor,
                                            mu_0=mu_0, L=L, showProgress=False)
    # RECOVERY KRONECKER
    recovered_signal_kron = kron_recovery(Y=Y, sigma_min=sigma_min, Phi=Phi, sigma_decrease_factor=sigma_decrease_factor,
                                            mu_0=mu_0, L=L, showProgress=False, kron_factor=kron_factor)
    

    ## EVALUATION
    # ----------------------------------------------------------------
    # Plot and SNR
    plot_signals(signal, recovered_signal, original_name="Original Signal", reconstructed_name="Recovered Signal")
    plot_signals(signal, recovered_signal_kron, original_name="Original Signal", reconstructed_name="Recovered Signal (Kronecker)")
